In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sn 
from seaborn import distplot
from seaborn import heatmap
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import math
import time
import numpy as np
from tqdm import tqdm
import multiprocessing
import pickle
from math import isinf 
from scipy.interpolate import interp1d
from numpy import array, zeros, full, argmin, inf, ndim
from sklearn import preprocessing


import statistics
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier as RB
#from sklearn.decomposition import KernelPCA
#from imblearn.over_sampling import SMOTE as SM
from sklearn.ensemble import RandomForestClassifier as RF,GradientBoostingRegressor as GB,ExtraTreesClassifier as ET
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neural_network import MLPClassifier as MP

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow import keras
from keras.layers import Dense, Dropout, Input, BatchNormalization
from keras import regularizers
from keras.models import Sequential, Model
from keras.callbacks import History 
history = History()

import matplotlib.pyplot as plt
from keras.layers import Dense,Activation,Layer,Lambda

import sklearn
from sklearn import preprocessing

 
# Scikit-Learn imports
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [ ]:
x_train,x_test,y_train,y_test= train_test_split(X_res, y_res,test_size = 0.5, random_state = 0) #try for test_size = 0.2, 0.3, 0.4

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
input_data = Input(shape=(1034,))#0
encoded = Dense(512, activation='relu')(input_data)#1
encoded = BatchNormalization()(encoded)#2
encoded = Dense(256, activation='relu')(encoded)#3
encoded = BatchNormalization()(encoded)#4
encoded = Dense(128, activation='relu')(encoded)#5
encoded = BatchNormalization()(encoded)#6

decoded = Dense(256, activation='sigmoid')(encoded)#7
decoded = BatchNormalization()(decoded)#8
decoded = Dense(512, activation='sigmoid')(decoded)#9
decoded = BatchNormalization()(decoded)#10
decoded = Dense(1034, activation='sigmoid')(decoded)#11

#del stackencoder
stackautoencoder = Model(input_data, decoded)
stackautoencoder.compile(loss="mse",optimizer='Adam')
stackautoencoder.summary()

In [ ]:
#Autoencoder1
input_data1 = Input(shape=(1034,))#0
encoded1 = Dense(512, activation='relu')(input_data1)#1
encoded1 = BatchNormalization()(encoded1)#2
decoded1 = Dense(1034, activation='sigmoid')(encoded1)#11

autoencoder1 = Model(input_data1, decoded1)
encoder1 = Model(input_data1, encoded1)

#Autoencoder2
input_data2 = Input(shape=(512,))
encoded2 = Dense(256, activation='relu')(input_data2)#3
encoded2 = BatchNormalization()(encoded2)#4
decoded2 = Dense(512, activation='relu')(encoded2)#9
decoded2 = BatchNormalization()(decoded2)#10

autoencoder2 = Model(input_data2, decoded2)
encoder2 = Model(input_data2, encoded2)

#Autoencoder3
input_data3 = Input(shape=(256,))
encoded3 = Dense(128, activation='relu')(input_data3)#5
encoded3 = BatchNormalization()(encoded3)#6
decoded3 = Dense(256, activation='relu')(encoded3)#7
decoded3 = BatchNormalization()(decoded3)#8

autoencoder3 = Model(input_data3, decoded3)
encoder3 = Model(input_data3, encoded3)

In [ ]:
autoencoder1.compile(loss="mse",optimizer='Adam')
autoencoder2.compile(loss="mse",optimizer='Adam')
autoencoder3.compile(loss="mse",optimizer='Adam')

encoder1.compile(loss="mse",optimizer='Adam')
encoder2.compile(loss="mse",optimizer='Adam')
encoder3.compile(loss="mse",optimizer='Adam')

In [ ]:
history1 = autoencoder1.fit(x_train, x_train,epochs=200, batch_size=512,shuffle=True,validation_split = 0.30)

In [ ]:
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.savefig('stackAE1_model_loss.png')
plt.show()

In [ ]:
first_layer_code = encoder1.predict(x_train)
print(first_layer_code.shape)

In [ ]:
history2 = autoencoder2.fit(first_layer_code, first_layer_code,epochs=200,batch_size=512,shuffle=True, validation_split = 0.30)

In [ ]:
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.savefig('stackAE2_model_loss.png')
plt.show()

In [ ]:
second_layer_code = encoder2.predict(first_layer_code)
print(first_layer_code.shape)

In [ ]:
history3 = autoencoder3.fit(second_layer_code, second_layer_code,epochs=800,batch_size=512,shuffle=True,validation_split = 0.30)

In [ ]:
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.savefig('stackAE3_model_loss.png')
plt.show()

In [ ]:
plt.plot(history1.history['loss'],'k',label='AE1 loss')
plt.plot(history1.history['val_loss'],'g',label='AE1 val loss')
plt.plot(history2.history['loss'],'y',label='AE2 loss')
plt.plot(history2.history['val_loss'],'m',label='AE2 val loss')
plt.plot(history3.history['loss'],'r',label='AE3 loss')
plt.plot(history3.history['val_loss'],'b',label='AE3 val loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='upper right')
plt.savefig('stackAEwhole_model_loss.png')
plt.show()

In [ ]:
stackautoencoder.layers[1].set_weights(autoencoder1.layers[1].get_weights()) # first dense layer
stackautoencoder.layers[2].set_weights(autoencoder1.layers[2].get_weights())
stackautoencoder.layers[3].set_weights(autoencoder2.layers[1].get_weights())
stackautoencoder.layers[4].set_weights(autoencoder2.layers[2].get_weights())
stackautoencoder.layers[5].set_weights(autoencoder3.layers[1].get_weights())
stackautoencoder.layers[6].set_weights(autoencoder3.layers[2].get_weights())
stackautoencoder.layers[7].set_weights(autoencoder3.layers[3].get_weights())
stackautoencoder.layers[8].set_weights(autoencoder3.layers[4].get_weights())
stackautoencoder.layers[9].set_weights(autoencoder2.layers[3].get_weights())
stackautoencoder.layers[10].set_weights(autoencoder2.layers[4].get_weights())
stackautoencoder.layers[11].set_weights(autoencoder1.layers[3].get_weights())

In [ ]:
evaluation = stackautoencoder.evaluate(x_test, x_test)
print("Loss:",evaluation)

In [ ]:
stackencoder = Model(input_data, encoded)
latent_vector_train = stackencoder.predict(x_train)
latent_vector = stackencoder.predict(x_test)

In [ ]:
latent_vector.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knnscores = []
for i in range(1,10):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(latent_vector_train, y_train)
    k = knn.score(latent_vector,y_test)
    knnscores.append(k)

s = np.array(knnscores)
result = np.where(s == s.max())
print('Knn Score : ',s.max(),'and k : ', result[0])

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

y_pred = knn.predict(latent_vector)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
latent_vector_train = np.load('/content/latent_vector_train.npy')
latent_vector = np.load('/content/latent_vector.npy')
y_train = np.load('/content/y_train.npy')
y_test = np.load('/content/y_test.npy')

In [ ]:
train_df = pd.DataFrame(latent_vector_train)
train_df['flag'] = y_train
test_df = pd.DataFrame(latent_vector)

In [ ]:
!pip install ctgan #https://sdv.dev/SDV/user_guides/single_table/ctgan.html

In [ ]:
from ctgan import CTGANSynthesizer
ctgan = CTGANSynthesizer()
ctgan.fit(train_df, discrete_columns=['flag'])

In [ ]:
samples = ctgan.sample(10000)

In [ ]:
ctgan_result_df = pd.concat([train_df,samples])
ctgan_result_df.head()

In [ ]:
y_train_ctgan = ctgan_result_df['flag']
X_train_ctgan = ctgan_result_df.drop('flag', axis=1)